# Exp 18

### compare models

In [1]:
import os
import sys
import datetime
import tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

import models
import report
import block_sampler
import batch_encoder
import callbacks
from dataset import Dataset
from report import Reporter
from trainer import Trainer
from attention import Attention, showSample
from batch_encoder import Dataset
from block_sampler import count_sectors, BlockSamplerByCategory, BlockSamplerFirstBlock
from batch_encoder import xs_encoder_8bits_11, BatchEncoder
from filter_random import gen_rndchk_models, evaluate_rnd_model, filter_dataset

from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Activation, TimeDistributed, Flatten, Dot, Softmax, Lambda, RepeatVector, Multiply, Permute, Reshape, BatchNormalization
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from trainer import TrainResults

/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/atila.alr/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
tensorflow.__version__, tensorflow.keras.__version__

('1.14.0', '2.2.4-tf')

In [3]:
tensorflow.test.is_gpu_available(), tensorflow.test.is_built_with_cuda()

(False, False)

In [24]:
# raw_dataset_folder='/home/user/sample200'
raw_dataset_folder='../datasets/govdocs1/sample200'
random_dataset_folder='../datasets/random'
minimum=200
maximum=200
result_dir = 'results/exp18b'
os.makedirs(result_dir, exist_ok=True)

In [5]:
def D(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Flatten()(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [6]:
def LD(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = LSTM(32)(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [7]:
def LL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = LSTM(32, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [8]:
def CD(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(64,(64,),strides=8)(last)
    last = Flatten()(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [9]:
def CM(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=1)(last)
    last = MaxPooling1D(pool_size=481, strides=1)(last)
    last = Flatten()(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [10]:
def CCM(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=1)(last)
    last = Conv1D(classes, (2,), strides=2)(last)
    last = MaxPooling1D(pool_size=240, strides=1)(last)
    last = Flatten()(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [11]:
def CL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(classes, (32,), strides=32)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [12]:
def CML(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(32, (32,), strides=32)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)    
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [13]:
def CLL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(32, (32,), strides=32)(last)
    last = LSTM(64, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [14]:
def CLD(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(256, (16,), strides=16)(last)
    last = LSTM(128)(last)
    last = Dense(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [15]:
def CCL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [16]:
def CCLL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = LSTM(64, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [17]:
def CMCML(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [18]:
def CMCMLL(classes, len_byte_vector):
    last = l0 = Input(shape=(512,len_byte_vector))
    last = Conv1D(128, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = Conv1D(64, (8,), strides=8)(last)
    last = MaxPooling1D(pool_size=2, strides=2, data_format='channels_first')(last)
    last = LSTM(64, return_sequences=True)(last)
    last = LSTM(classes)(last)
    last = Activation('softmax')(last)
    name = sys._getframe().f_code.co_name
    model = tf.keras.Model([l0], last, name=name)
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(),
        metrics=['binary_accuracy', 'categorical_accuracy'])
    return model

In [27]:
classes = 30
len_byte_vector=256
mymodels = [
    D(classes, len_byte_vector),

    CD(classes, len_byte_vector),
    CM(classes, len_byte_vector),
    CCM(classes, len_byte_vector),
    
    CL(classes, len_byte_vector),
    CML(classes, len_byte_vector),
    CLL(classes, len_byte_vector),
    CLD(classes, len_byte_vector),
    
    CCL(classes, len_byte_vector),
    CCLL(classes, len_byte_vector),
    CMCML(classes, len_byte_vector),
    CMCMLL(classes, len_byte_vector),

    LD(classes, len_byte_vector),
    LL(classes, len_byte_vector),
]

In [20]:
rawset = Dataset.new_from_folders(raw_dataset_folder).filter_min_max(minimum, maximum)

In [21]:
tset, vset = rawset.rnd_split_fraction_by_category(0.5)

In [25]:
class MyTrainer:
    def __init__(self,
                 model,
                 group_by='by_file',
                 xs_encoder='8bits_11',
                 validation_steps=10,
                 steps_per_epoch=10,
                 epochs=10000000,
                 max_seconds=10*60,
                 batch_size=400,
                 min_delta=1e-03,
                 patience=4,
                 blockSampler=block_sampler.BlockSampler,
                 batchEncoder=batch_encoder.BatchEncoder):
        self.model = model
        self.group_by = group_by
        self.xs_encoder = xs_encoder
        self.validation_steps = validation_steps
        self.steps_per_epoch = steps_per_epoch
        self.epochs = epochs
        self.max_seconds = max_seconds
        self.batch_size = batch_size
        self.min_delta = min_delta
        self.patience = patience
        self.blockSampler = blockSampler
        self.batchEncoder = batchEncoder

    def train(self, tset, vset):
        tsampler = self.blockSampler(tset, group_by=self.group_by)
        tbenc = self.batchEncoder(tsampler, self.batch_size,
                                  xs_encoder=self.xs_encoder)

        vsampler = self.blockSampler(vset, group_by=self.group_by)
        vbenc = self.batchEncoder(vsampler, self.batch_size,
                                  xs_encoder=self.xs_encoder)

        model = self.model

        timeIt = callbacks.TimeIt()

        history = model.fit_generator(iter(tbenc),
                                      validation_data=iter(vbenc),
                                      validation_steps=self.validation_steps,
                                      steps_per_epoch=self.steps_per_epoch,
                                      epochs=self.epochs,
                                      verbose=1,
                                      callbacks=[
            timeIt,
            # callbacks.SaveModel(os.path.join(result_dir, model.name + '.h5')),
#             callbacks.TimeLimit(self.max_seconds),
            EarlyStopping(monitor='val_categorical_accuracy',
                          min_delta=self.min_delta, patience=self.patience),
            TensorBoard(
                log_dir=os.path.join(result_dir, model.name),
            ),
        ],
            use_multiprocessing=False,
            workers=0,
        )
        return TrainResults(
            model=model,
            history=history,
            metrics=['val_binary_accuracy', 'val_categorical_accuracy'],
            elapsed=timeIt.elapsed,
        )

In [29]:
r = report.Reporter2(result_dir + '/models.tsv')
for model in mymodels:
    print(model.name)
    result = MyTrainer(
        model,
        xs_encoder='one_hot',
        batch_size=100,
        steps_per_epoch=30,
        validation_steps=30,
        patience=20,
    ).train(tset, vset)
    print(result.elapsed)
    r.line(name=result.model.name,
           elapsed=result.elapsed,
           **report.report_epochs(**result._asdict()),
           **report.report_metrics(result.history, result.metrics),
          )

r.close()

D
Epoch 1/10000000
30/30 [==============================] - 14s 476ms/step - loss: 2.8016 - binary_accuracy: 0.9675 - categorical_accuracy: 0.2317 - val_loss: 2.4114 - val_binary_accuracy: 0.9686 - val_categorical_accuracy: 0.3167
Epoch 2/10000000
30/30 [==============================] - 12s 405ms/step - loss: 2.1581 - binary_accuracy: 0.9706 - categorical_accuracy: 0.3720 - val_loss: 2.1416 - val_binary_accuracy: 0.9700 - val_categorical_accuracy: 0.3673
Epoch 3/10000000
30/30 [==============================] - 12s 412ms/step - loss: 1.9636 - binary_accuracy: 0.9724 - categorical_accuracy: 0.4320 - val_loss: 2.0584 - val_binary_accuracy: 0.9709 - val_categorical_accuracy: 0.3997
Epoch 4/10000000
30/30 [==============================] - 12s 408ms/step - loss: 1.8228 - binary_accuracy: 0.9739 - categorical_accuracy: 0.4560 - val_loss: 1.9703 - val_binary_accuracy: 0.9716 - val_categorical_accuracy: 0.4093
Epoch 5/10000000
30/30 [==============================] - 13s 417ms/step - loss: 1

Epoch 37/10000000
30/30 [==============================] - 13s 425ms/step - loss: 1.1751 - binary_accuracy: 0.9837 - categorical_accuracy: 0.6610 - val_loss: 1.8790 - val_binary_accuracy: 0.9742 - val_categorical_accuracy: 0.4503
Epoch 38/10000000
30/30 [==============================] - 12s 415ms/step - loss: 1.0978 - binary_accuracy: 0.9844 - categorical_accuracy: 0.6673 - val_loss: 1.8499 - val_binary_accuracy: 0.9747 - val_categorical_accuracy: 0.4557
Epoch 39/10000000
30/30 [==============================] - 12s 414ms/step - loss: 1.1459 - binary_accuracy: 0.9839 - categorical_accuracy: 0.6573 - val_loss: 1.8384 - val_binary_accuracy: 0.9750 - val_categorical_accuracy: 0.4610
Epoch 40/10000000
30/30 [==============================] - 12s 414ms/step - loss: 1.1390 - binary_accuracy: 0.9838 - categorical_accuracy: 0.6603 - val_loss: 1.8841 - val_binary_accuracy: 0.9740 - val_categorical_accuracy: 0.4577
Epoch 41/10000000
30/30 [==============================] - 13s 424ms/step - loss

Epoch 73/10000000
30/30 [==============================] - 12s 413ms/step - loss: 1.0010 - binary_accuracy: 0.9861 - categorical_accuracy: 0.7087 - val_loss: 1.9340 - val_binary_accuracy: 0.9730 - val_categorical_accuracy: 0.4487
Epoch 74/10000000
30/30 [==============================] - 13s 417ms/step - loss: 0.9596 - binary_accuracy: 0.9868 - categorical_accuracy: 0.7237 - val_loss: 1.9954 - val_binary_accuracy: 0.9739 - val_categorical_accuracy: 0.4573
Epoch 75/10000000
30/30 [==============================] - 12s 416ms/step - loss: 0.9597 - binary_accuracy: 0.9865 - categorical_accuracy: 0.7190 - val_loss: 1.9752 - val_binary_accuracy: 0.9731 - val_categorical_accuracy: 0.4467
Epoch 76/10000000
30/30 [==============================] - 12s 415ms/step - loss: 0.9947 - binary_accuracy: 0.9862 - categorical_accuracy: 0.7143 - val_loss: 2.0266 - val_binary_accuracy: 0.9727 - val_categorical_accuracy: 0.4323
Epoch 77/10000000
30/30 [==============================] - 13s 418ms/step - loss

30/30 [==============================] - 20s 651ms/step - loss: 1.2351 - binary_accuracy: 0.9818 - categorical_accuracy: 0.6217 - val_loss: 1.9631 - val_binary_accuracy: 0.9737 - val_categorical_accuracy: 0.4553
Epoch 30/10000000
30/30 [==============================] - 20s 654ms/step - loss: 1.3165 - binary_accuracy: 0.9809 - categorical_accuracy: 0.6160 - val_loss: 1.8805 - val_binary_accuracy: 0.9745 - val_categorical_accuracy: 0.4623
Epoch 31/10000000
30/30 [==============================] - 20s 655ms/step - loss: 1.2721 - binary_accuracy: 0.9812 - categorical_accuracy: 0.6273 - val_loss: 1.9271 - val_binary_accuracy: 0.9749 - val_categorical_accuracy: 0.4743
Epoch 32/10000000
30/30 [==============================] - 19s 643ms/step - loss: 1.2757 - binary_accuracy: 0.9816 - categorical_accuracy: 0.6167 - val_loss: 1.8774 - val_binary_accuracy: 0.9745 - val_categorical_accuracy: 0.4707
Epoch 33/10000000
30/30 [==============================] - 19s 648ms/step - loss: 1.2411 - binary_

Epoch 28/10000000
30/30 [==============================] - 37s 1s/step - loss: 1.6456 - binary_accuracy: 0.9753 - categorical_accuracy: 0.5637 - val_loss: 1.8000 - val_binary_accuracy: 0.9741 - val_categorical_accuracy: 0.4690
Epoch 29/10000000
30/30 [==============================] - 37s 1s/step - loss: 1.6257 - binary_accuracy: 0.9752 - categorical_accuracy: 0.5700 - val_loss: 1.8043 - val_binary_accuracy: 0.9741 - val_categorical_accuracy: 0.4797
Epoch 30/10000000
30/30 [==============================] - 37s 1s/step - loss: 1.5961 - binary_accuracy: 0.9758 - categorical_accuracy: 0.5797 - val_loss: 1.7770 - val_binary_accuracy: 0.9746 - val_categorical_accuracy: 0.4817
Epoch 31/10000000
30/30 [==============================] - 38s 1s/step - loss: 1.5741 - binary_accuracy: 0.9759 - categorical_accuracy: 0.5847 - val_loss: 1.7800 - val_binary_accuracy: 0.9746 - val_categorical_accuracy: 0.4763
Epoch 32/10000000
30/30 [==============================] - 37s 1s/step - loss: 1.5695 - bina

...

Epoch 54/10000000
30/30 [==============================] - 13s 432ms/step - loss: 2.3170 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3963 - val_loss: 2.3690 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3873
Epoch 55/10000000
30/30 [==============================] - 13s 436ms/step - loss: 2.3137 - binary_accuracy: 0.9667 - categorical_accuracy: 0.3960 - val_loss: 2.3651 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3787
Epoch 56/10000000
30/30 [==============================] - 13s 436ms/step - loss: 2.3179 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4010 - val_loss: 2.3502 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3983
Epoch 57/10000000
30/30 [==============================] - 13s 436ms/step - loss: 2.2968 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4107 - val_loss: 2.3522 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.3903
Epoch 58/10000000
30/30 [==============================] - 13s 438ms/step - loss

Epoch 90/10000000
30/30 [==============================] - 13s 442ms/step - loss: 2.2173 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4493 - val_loss: 2.2913 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.4247
Epoch 91/10000000
30/30 [==============================] - 13s 443ms/step - loss: 2.2187 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4207 - val_loss: 2.2950 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.4050
Epoch 92/10000000
30/30 [==============================] - 13s 443ms/step - loss: 2.2248 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4300 - val_loss: 2.3044 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.4020
Epoch 93/10000000
30/30 [==============================] - 13s 444ms/step - loss: 2.2107 - binary_accuracy: 0.9667 - categorical_accuracy: 0.4423 - val_loss: 2.3004 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.4093
Epoch 94/10000000
30/30 [==============================] - 13s 443ms/step - loss

30/30 [==============================] - 27s 905ms/step - loss: 2.8193 - binary_accuracy: 0.9671 - categorical_accuracy: 0.1193 - val_loss: 2.7412 - val_binary_accuracy: 0.9671 - val_categorical_accuracy: 0.1387
Epoch 23/10000000
30/30 [==============================] - 27s 908ms/step - loss: 2.6684 - binary_accuracy: 0.9673 - categorical_accuracy: 0.1733 - val_loss: 2.6110 - val_binary_accuracy: 0.9673 - val_categorical_accuracy: 0.1963
Epoch 24/10000000
30/30 [==============================] - 27s 893ms/step - loss: 2.6017 - binary_accuracy: 0.9672 - categorical_accuracy: 0.1883 - val_loss: 2.6137 - val_binary_accuracy: 0.9670 - val_categorical_accuracy: 0.1873
Epoch 25/10000000
30/30 [==============================] - 27s 896ms/step - loss: 2.5732 - binary_accuracy: 0.9672 - categorical_accuracy: 0.1917 - val_loss: 2.5557 - val_binary_accuracy: 0.9674 - val_categorical_accuracy: 0.1913
Epoch 26/10000000
30/30 [==============================] - 27s 903ms/step - loss: 2.5168 - binary_

Epoch 58/10000000
30/30 [==============================] - 27s 890ms/step - loss: 2.6492 - binary_accuracy: 0.9670 - categorical_accuracy: 0.1670 - val_loss: 2.6657 - val_binary_accuracy: 0.9670 - val_categorical_accuracy: 0.1727
Epoch 59/10000000
30/30 [==============================] - 27s 898ms/step - loss: 2.6116 - binary_accuracy: 0.9672 - categorical_accuracy: 0.1930 - val_loss: 2.6175 - val_binary_accuracy: 0.9671 - val_categorical_accuracy: 0.1973
Epoch 60/10000000
30/30 [==============================] - 27s 897ms/step - loss: 2.5768 - binary_accuracy: 0.9671 - categorical_accuracy: 0.2123 - val_loss: 2.5891 - val_binary_accuracy: 0.9670 - val_categorical_accuracy: 0.1983
Epoch 61/10000000
30/30 [==============================] - 27s 892ms/step - loss: 2.5458 - binary_accuracy: 0.9671 - categorical_accuracy: 0.2067 - val_loss: 2.5814 - val_binary_accuracy: 0.9671 - val_categorical_accuracy: 0.1940
Epoch 62/10000000
30/30 [==============================] - 27s 898ms/step - loss

Epoch 19/10000000
30/30 [==============================] - 43s 1s/step - loss: 3.0335 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1103 - val_loss: 3.1114 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0753
Epoch 20/10000000
30/30 [==============================] - 42s 1s/step - loss: 3.0814 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0820 - val_loss: 3.0901 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0933
Epoch 21/10000000
30/30 [==============================] - 42s 1s/step - loss: 3.1462 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0807 - val_loss: 3.2695 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0650
Epoch 22/10000000
30/30 [==============================] - 42s 1s/step - loss: 3.3234 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0500 - val_loss: 3.1626 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0703
Epoch 23/10000000
30/30 [==============================] - 42s 1s/step - loss: 3.1066 - bina

In [30]:
result_dir = 'results/exp18b-patience60'
os.makedirs(result_dir, exist_ok=True)
r = report.Reporter2(result_dir + '/models.tsv')
for model in mymodels:
    print(model.name)
    result = MyTrainer(
        model,
        xs_encoder='one_hot',
        batch_size=100,
        steps_per_epoch=30,
        validation_steps=30,
        patience=60,
    ).train(tset, vset)
    print(result.elapsed)
    r.line(name=result.model.name,
           elapsed=result.elapsed,
           **report.report_epochs(**result._asdict()),
           **report.report_metrics(result.history, result.metrics),
          )

r.close()

D
Epoch 1/10000000
30/30 [==============================] - 12s 392ms/step - loss: 0.9414 - binary_accuracy: 0.9865 - categorical_accuracy: 0.7273 - val_loss: 1.9884 - val_binary_accuracy: 0.9736 - val_categorical_accuracy: 0.4550
Epoch 2/10000000
30/30 [==============================] - 12s 387ms/step - loss: 0.9493 - binary_accuracy: 0.9867 - categorical_accuracy: 0.7300 - val_loss: 1.9720 - val_binary_accuracy: 0.9734 - val_categorical_accuracy: 0.4580
Epoch 3/10000000
30/30 [==============================] - 12s 395ms/step - loss: 0.9756 - binary_accuracy: 0.9863 - categorical_accuracy: 0.7200 - val_loss: 1.9671 - val_binary_accuracy: 0.9732 - val_categorical_accuracy: 0.4463
Epoch 4/10000000
30/30 [==============================] - 12s 399ms/step - loss: 0.8997 - binary_accuracy: 0.9877 - categorical_accuracy: 0.7440 - val_loss: 1.9596 - val_binary_accuracy: 0.9740 - val_categorical_accuracy: 0.4643
Epoch 5/10000000
30/30 [==============================] - 12s 409ms/step - loss: 0

Epoch 37/10000000
30/30 [==============================] - 12s 396ms/step - loss: 0.8733 - binary_accuracy: 0.9876 - categorical_accuracy: 0.7637 - val_loss: 2.0136 - val_binary_accuracy: 0.9728 - val_categorical_accuracy: 0.4533
Epoch 38/10000000
30/30 [==============================] - 12s 398ms/step - loss: 0.8726 - binary_accuracy: 0.9878 - categorical_accuracy: 0.7543 - val_loss: 2.0637 - val_binary_accuracy: 0.9725 - val_categorical_accuracy: 0.4513
Epoch 39/10000000
30/30 [==============================] - 12s 398ms/step - loss: 0.8331 - binary_accuracy: 0.9876 - categorical_accuracy: 0.7597 - val_loss: 2.1304 - val_binary_accuracy: 0.9716 - val_categorical_accuracy: 0.4460
Epoch 40/10000000
30/30 [==============================] - 12s 398ms/step - loss: 0.8612 - binary_accuracy: 0.9875 - categorical_accuracy: 0.7603 - val_loss: 2.0807 - val_binary_accuracy: 0.9718 - val_categorical_accuracy: 0.4367
Epoch 41/10000000
30/30 [==============================] - 12s 398ms/step - loss

30/30 [==============================] - 19s 642ms/step - loss: 1.1999 - binary_accuracy: 0.9823 - categorical_accuracy: 0.6530 - val_loss: 1.9771 - val_binary_accuracy: 0.9734 - val_categorical_accuracy: 0.4567
Epoch 9/10000000
30/30 [==============================] - 19s 640ms/step - loss: 1.1749 - binary_accuracy: 0.9827 - categorical_accuracy: 0.6553 - val_loss: 1.9846 - val_binary_accuracy: 0.9734 - val_categorical_accuracy: 0.4533
Epoch 10/10000000
30/30 [==============================] - 19s 638ms/step - loss: 1.1792 - binary_accuracy: 0.9825 - categorical_accuracy: 0.6517 - val_loss: 1.9842 - val_binary_accuracy: 0.9730 - val_categorical_accuracy: 0.4483
Epoch 11/10000000
30/30 [==============================] - 19s 637ms/step - loss: 1.1436 - binary_accuracy: 0.9833 - categorical_accuracy: 0.6610 - val_loss: 1.9104 - val_binary_accuracy: 0.9745 - val_categorical_accuracy: 0.4693
Epoch 12/10000000
30/30 [==============================] - 19s 638ms/step - loss: 1.2005 - binary_a

Epoch 44/10000000
30/30 [==============================] - 19s 641ms/step - loss: 1.1063 - binary_accuracy: 0.9841 - categorical_accuracy: 0.6867 - val_loss: 2.0695 - val_binary_accuracy: 0.9730 - val_categorical_accuracy: 0.4530
Epoch 45/10000000
30/30 [==============================] - 19s 641ms/step - loss: 1.1151 - binary_accuracy: 0.9838 - categorical_accuracy: 0.6710 - val_loss: 2.0887 - val_binary_accuracy: 0.9731 - val_categorical_accuracy: 0.4610
Epoch 46/10000000
30/30 [==============================] - 19s 643ms/step - loss: 1.0934 - binary_accuracy: 0.9838 - categorical_accuracy: 0.6870 - val_loss: 2.1481 - val_binary_accuracy: 0.9722 - val_categorical_accuracy: 0.4403
Epoch 47/10000000
30/30 [==============================] - 19s 641ms/step - loss: 1.0685 - binary_accuracy: 0.9839 - categorical_accuracy: 0.6863 - val_loss: 2.1261 - val_binary_accuracy: 0.9725 - val_categorical_accuracy: 0.4503
Epoch 48/10000000
30/30 [==============================] - 19s 644ms/step - loss

Epoch 80/10000000
30/30 [==============================] - 19s 641ms/step - loss: 1.0349 - binary_accuracy: 0.9845 - categorical_accuracy: 0.7057 - val_loss: 2.2476 - val_binary_accuracy: 0.9714 - val_categorical_accuracy: 0.4300
Epoch 81/10000000
30/30 [==============================] - 19s 644ms/step - loss: 1.0394 - binary_accuracy: 0.9845 - categorical_accuracy: 0.7010 - val_loss: 2.2343 - val_binary_accuracy: 0.9720 - val_categorical_accuracy: 0.4460
Epoch 82/10000000
30/30 [==============================] - 19s 643ms/step - loss: 1.0632 - binary_accuracy: 0.9841 - categorical_accuracy: 0.6977 - val_loss: 2.2226 - val_binary_accuracy: 0.9719 - val_categorical_accuracy: 0.4447
Epoch 83/10000000
30/30 [==============================] - 19s 643ms/step - loss: 1.0654 - binary_accuracy: 0.9840 - categorical_accuracy: 0.6963 - val_loss: 2.2559 - val_binary_accuracy: 0.9723 - val_categorical_accuracy: 0.4587
Epoch 84/10000000
30/30 [==============================] - 20s 651ms/step - loss

...

30/30 [==============================] - 27s 893ms/step - loss: 2.0782 - binary_accuracy: 0.9687 - categorical_accuracy: 0.3323 - val_loss: 2.1439 - val_binary_accuracy: 0.9687 - val_categorical_accuracy: 0.3120
Epoch 162/10000000
30/30 [==============================] - 27s 892ms/step - loss: 2.0797 - binary_accuracy: 0.9686 - categorical_accuracy: 0.3190 - val_loss: 2.1273 - val_binary_accuracy: 0.9688 - val_categorical_accuracy: 0.3070
Epoch 163/10000000
30/30 [==============================] - 27s 902ms/step - loss: 2.0645 - binary_accuracy: 0.9689 - categorical_accuracy: 0.3253 - val_loss: 2.1190 - val_binary_accuracy: 0.9685 - val_categorical_accuracy: 0.3120
Epoch 164/10000000
30/30 [==============================] - 27s 896ms/step - loss: 2.1975 - binary_accuracy: 0.9688 - categorical_accuracy: 0.3113 - val_loss: 2.4053 - val_binary_accuracy: 0.9681 - val_categorical_accuracy: 0.2737
Epoch 165/10000000
30/30 [==============================] - 26s 879ms/step - loss: 2.3455 - bin

Epoch 19/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9990 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0910 - val_loss: 3.0068 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0980
Epoch 20/10000000
30/30 [==============================] - 43s 1s/step - loss: 2.9864 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1007 - val_loss: 2.9970 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1053
Epoch 21/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9945 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1097 - val_loss: 3.0298 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1013
Epoch 22/10000000
30/30 [==============================] - 42s 1s/step - loss: 3.0393 - binary_accuracy: 0.9667 - categorical_accuracy: 0.0907 - val_loss: 2.9885 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1020
Epoch 23/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9870 - bina

30/30 [==============================] - 43s 1s/step - loss: 2.9444 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1213 - val_loss: 2.9760 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1073
Epoch 56/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9673 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1110 - val_loss: 2.9679 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1160
Epoch 57/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9600 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1117 - val_loss: 2.9514 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1220
Epoch 58/10000000
30/30 [==============================] - 43s 1s/step - loss: 2.9352 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1257 - val_loss: 2.9626 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1247
Epoch 59/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9443 - binary_accuracy: 0.966

30/30 [==============================] - 42s 1s/step - loss: 3.0323 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1030 - val_loss: 3.0878 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.0960
Epoch 92/10000000
30/30 [==============================] - 43s 1s/step - loss: 3.0294 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1017 - val_loss: 3.0187 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1050
Epoch 93/10000000
30/30 [==============================] - 42s 1s/step - loss: 3.0122 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1027 - val_loss: 3.0086 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1057
Epoch 94/10000000
30/30 [==============================] - 43s 1s/step - loss: 3.0011 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1053 - val_loss: 3.0108 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1020
Epoch 95/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.9907 - binary_accuracy: 0.966

30/30 [==============================] - 42s 1s/step - loss: 2.8295 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1390 - val_loss: 2.8514 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1323
Epoch 128/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.8547 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1177 - val_loss: 2.8513 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1263
Epoch 129/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.8338 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1347 - val_loss: 2.8309 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1513
Epoch 130/10000000
30/30 [==============================] - 42s 1s/step - loss: 2.8522 - binary_accuracy: 0.9667 - categorical_accuracy: 0.1227 - val_loss: 2.8396 - val_binary_accuracy: 0.9667 - val_categorical_accuracy: 0.1337
5510.025587320328
